In [1]:
import pickle 
import networkx as nx
import pandas as pd
import sqlite3

In [3]:
def load_pickle(fname):
    with open(fname, 'rb') as f:
        return pickle.load(f)

G = load_pickle('./MulDiGraph.pkl')

In [5]:
conn = sqlite3.connect('graph.db')

## Read nodes and edges

In [4]:
nodes = []
# for idx, nd in enumerate(nx.nodes(G)):
#     row = [idx, nd, G.nodes[nd]['isp']]
#     print(nd, idx, G.nodes[nd]['isp'])
#     nodes.append(row)

edges = []
for ind, edge in enumerate(nx.edges(G)):
    (u, v) = edge
    eg = G[u][v][0]
    amo, tim = eg['amount'], eg['timestamp']
    item = [amo, tim, u, v]
    edges.append(item)

In [6]:
# nodes_df = pd.DataFrame(nodes, columns=['idx', 'node', 'isp'])
nodes_df = pd.read_sql_query("SELECT * FROM nodes_with_degree_pagerank", conn)

In [37]:
test_set = set()
lst = list(G.in_edges(["0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb"]))
for pair in lst:
    print(pair)
    test_set.add(pair[x] for x in [0,1])

('0x1266f8b9e4dffc9e2f719bf51713f7e714516861', '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb')
('0xa7acac8c1942873f934dbad78910a20ad774588b', '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb')
('0x210ab95cac537f539d3455db57f1e5072befde52', '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb')
('0x3ae74b9f78837faa1527b296f1940bd6f4e15bde', '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb')
('0x1522900b6dafac587d499a862861c0869be6e428', '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb')
('0x1522900b6dafac587d499a862861c0869be6e428', '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb')
('0xff4171e9fba22a6df8b3cd26fb7dd360f15f3583', '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb')
('0x86e6df2933f88a00caa11ad19413123c3abef3de', '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb')


In [64]:
list(G.in_edges(["0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb"]))

[('0x1266f8b9e4dffc9e2f719bf51713f7e714516861',
  '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb'),
 ('0xa7acac8c1942873f934dbad78910a20ad774588b',
  '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb'),
 ('0x210ab95cac537f539d3455db57f1e5072befde52',
  '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb'),
 ('0x3ae74b9f78837faa1527b296f1940bd6f4e15bde',
  '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb'),
 ('0x1522900b6dafac587d499a862861c0869be6e428',
  '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb'),
 ('0x1522900b6dafac587d499a862861c0869be6e428',
  '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb'),
 ('0xff4171e9fba22a6df8b3cd26fb7dd360f15f3583',
  '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb'),
 ('0x86e6df2933f88a00caa11ad19413123c3abef3de',
  '0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb')]

In [67]:
nodes_to_include = list(nodes_set)

In [68]:
len(nodes_to_include)

29461

In [69]:
sub_G = G.subgraph(nodes_to_include)

In [74]:
G_in_degree = dict(G.in_degree())

In [81]:
G_out_degree = dict(G.out_degree())

In [78]:
for index, row in nodes_df.iterrows():
    nodes_df.loc[index, "in_degree"] = G_in_degree[row["node"]]

In [82]:
for index, row in nodes_df.iterrows():
    nodes_df.loc[index, "out_degree"] = G_out_degree[row["node"]]

In [84]:
nodes_df.head()

,idx,node,isp,in_degree,out_degree
0,0,0x1f1e784a61a8ca0a90250bcd2170696655b28a21,0,215.0,1624.0
1,1,0x1266f8b9e4dffc9e2f719bf51713f7e714516861,0,1.0,1.0
2,2,0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb,1,8.0,0.0
3,3,0x256fc19e9d8f5be0d451841f218289d1adbbaaa3,0,0.0,1.0
4,4,0xb50d0c4cb2c29cc232c96a59e9c65eb82914ec75,0,63.0,166.0


In [86]:
nodes_df.to_sql('nodes_with_degree', sqlite3.connect('graph.db'), if_exists='replace')

2973489

In [17]:
edges_df = pd.DataFrame(edges, columns=['amount', 'timestamp', 'source', 'target'])

In [87]:
pageranks = nx.pagerank(G)

In [89]:
for index, row in nodes_df.iterrows():
    nodes_df.loc[index, "pagerank"] = pageranks[row["node"]]

In [90]:
nodes_df.head()

,idx,node,isp,in_degree,out_degree,pagerank
0,0,0x1f1e784a61a8ca0a90250bcd2170696655b28a21,0,215.0,1624.0,6.168217e-07
1,1,0x1266f8b9e4dffc9e2f719bf51713f7e714516861,0,1.0,1.0,1.333369e-07
2,2,0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb,1,8.0,0.0,8.449769e-07
3,3,0x256fc19e9d8f5be0d451841f218289d1adbbaaa3,0,0.0,1.0,1.331609e-07
4,4,0xb50d0c4cb2c29cc232c96a59e9c65eb82914ec75,0,63.0,166.0,8.827056e-07


In [91]:
nodes_df.to_sql('nodes_with_degree_pagerank', sqlite3.connect('graph.db'), if_exists='replace')

2973489

In [19]:
query = f"""
SELECT SUM(amount) as weightsout, source
FROM edges
GROUP BY source
"""

G_out_weights = pd.read_sql_query(query, conn)

In [20]:
G_out_weights.head()

,SUM(amount),source
0,28.98,0x0000000008b4c94610f112d6bae67a198ca6269d
1,0.20,0x000000004fdd41501a42651743c55f7e41ec85e4
2,0.00,0x0000000050514c83aa5d55cb5dcc774fc69641c7
3,0.30,0x00000000ad4eb06bc550dc45ce28888c0626839a
4,0.00,0x00000000b1ad45d63cf1461e8b2b2e11c27dce8a


In [21]:
query = f"""
SELECT SUM(amount) as weightsin, target
FROM edges
GROUP BY target
"""

G_in_weights = pd.read_sql_query(query, conn)

In [22]:
G_in_weights.head()

,SUM(amount),target
0,1.763198e+03,0x0000000000000000000000000000000000000000
1,1.000000e-16,0x0000000000000000000000000000000000000001
2,0.000000e+00,0x0000000000000000000000000000000000000002
3,0.000000e+00,0x0000000000000000000000000000000000002799
4,4.000000e-01,0x000000000000000000000000000000000000570d


In [23]:
nodes_df.head()

,index,idx,node,isp,in_degree,out_degree,pagerank
0,0,0,0x1f1e784a61a8ca0a90250bcd2170696655b28a21,0,215.0,1624.0,6.168217e-07
1,1,1,0x1266f8b9e4dffc9e2f719bf51713f7e714516861,0,1.0,1.0,1.333369e-07
2,2,2,0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb,1,8.0,0.0,8.449769e-07
3,3,3,0x256fc19e9d8f5be0d451841f218289d1adbbaaa3,0,0.0,1.0,1.331609e-07
4,4,4,0xb50d0c4cb2c29cc232c96a59e9c65eb82914ec75,0,63.0,166.0,8.827056e-07


In [24]:
nodes_df_with_weights = nodes_df.merge(G_in_weights, left_on='node', right_on='source', how='left')
nodes_df_with_weights.drop(columns=['source'], inplace=True)
nodes_df_with_weights.drop(columns=['index'], inplace=True)

In [33]:
nodes_df_with_weights = nodes_df_with_weights.merge(G_out_weights, left_on='node', right_on='target', how='left')
nodes_df_with_weights.drop(columns=['target'], inplace=True)

In [45]:
nodes_df_with_weights.columns = ['idx', 'node', 'isp', 'in_degree', 'out_degree', 'pagerank', 'weightsout', 'weightsin']

In [46]:
query = f"""
SELECT COUNT(*) as numin, source
FROM edges
GROUP BY source
"""

G_out_num = pd.read_sql_query(query, conn)

In [47]:
query = f"""
SELECT COUNT(*) as numout, target
FROM edges
GROUP BY target
"""

G_in_num = pd.read_sql_query(query, conn)

In [48]:
G_out_num.head()

,numin,source
0,4,0x0000000008b4c94610f112d6bae67a198ca6269d
1,2,0x000000004fdd41501a42651743c55f7e41ec85e4
2,1,0x0000000050514c83aa5d55cb5dcc774fc69641c7
3,1,0x00000000ad4eb06bc550dc45ce28888c0626839a
4,2,0x00000000b1ad45d63cf1461e8b2b2e11c27dce8a


In [49]:
G_in_num.head()

,numout,target
0,16,0x0000000000000000000000000000000000000000
1,4,0x0000000000000000000000000000000000000001
2,1,0x0000000000000000000000000000000000000002
3,1,0x0000000000000000000000000000000000002799
4,1,0x000000000000000000000000000000000000570d


In [50]:
nodes_df_with_weights_num = nodes_df_with_weights.merge(G_in_num, left_on='node', right_on='target', how='left')
nodes_df_with_weights_num.drop(columns=['target'], inplace=True)
nodes_df_with_weights_num = nodes_df_with_weights_num.merge(G_out_num, left_on='node', right_on='source', how='left')
nodes_df_with_weights_num.drop(columns=['source'], inplace=True)

In [52]:
nodes_df_with_weights_num.fillna(0, inplace=True)

In [53]:
nodes_df_with_weights_num.to_sql('draft_features', sqlite3.connect('graph.db'), if_exists='replace')

2973489

In [54]:
nodes_df_with_weights_num.shape

(2973489, 10)

In [55]:
nodes_df_with_weights_num.head()

,idx,node,isp,in_degree,out_degree,pagerank,weightsout,weightsin,numout,numin
0,0,0x1f1e784a61a8ca0a90250bcd2170696655b28a21,0,215.0,1624.0,6.168217e-07,14285.789338,15931.196110,215.0,1624.0
1,1,0x1266f8b9e4dffc9e2f719bf51713f7e714516861,0,1.0,1.0,1.333369e-07,2.344203,2.344623,1.0,1.0
2,2,0xbbfaf27674c2eb5d13edc58a40081248d13dcfeb,1,8.0,0.0,8.449769e-07,0.000000,25.268098,8.0,0.0
3,3,0x256fc19e9d8f5be0d451841f218289d1adbbaaa3,0,0.0,1.0,1.331609e-07,0.489485,0.000000,0.0,1.0
4,4,0xb50d0c4cb2c29cc232c96a59e9c65eb82914ec75,0,63.0,166.0,8.827056e-07,19.336007,8.408347,63.0,166.0


## Test with Training

In [70]:
fraud = nodes_df_with_weights_num[nodes_df_with_weights_num['isp'] == 1]
fraud.shape

(1165, 10)

In [72]:
nodes_set = set()
for index, row in fraud.iterrows():
    in_edges = list(G.in_edges([row["node"]]))
    out_edges = list(G.out_edges([row["node"]]))
    nodes_set.add(row["node"])
    for pair in in_edges:
        nodes_set.add(pair[0])
    for pair in out_edges:
        nodes_set.add(pair[1])

In [73]:
nodes_to_include = list(nodes_set)

In [76]:
nodes_to_include = pd.DataFrame(nodes_to_include, columns=['node'])
nodes_to_include = nodes_to_include.merge(nodes_df_with_weights_num, left_on='node', right_on='node', how='left')

In [79]:
nodes_to_include.head()

,node,idx,isp,in_degree,out_degree,pagerank,weightsout,weightsin,numout,numin
0,0xe5d0f67cf53066430ca3e3fcedfbe387500c24f1,766752,0,1.0,7.0,1.333872e-07,0.000000,0.001400,1.0,7.0
1,0xe6c76bb82397d4e5ef0ebf58133dcbef5b1d70c5,64503,0,1.0,1.0,4.190203e-07,0.014891,0.015038,1.0,1.0
2,0x0f5662bebf0315b659e8b9ceeae92ea628aceae6,1991399,0,6.0,9.0,4.191195e-07,14.266521,17.627004,6.0,9.0
3,0xb0f6ec1275d458aae0d94314cede57bf47558751,157383,0,1.0,1.0,4.190203e-07,0.500000,0.500126,1.0,1.0
4,0x444c4332c6e1e26ed67aac6da5f83bc5ff450beb,682817,0,4.0,7.0,2.060771e-07,0.146212,0.151640,4.0,7.0


In [87]:
nodes_to_include.to_csv('nodes_to_include.csv', index=False)

In [80]:
X = nodes_to_include.drop(columns=['node', 'isp', 'idx'])
y = nodes_to_include['isp']

In [81]:
## Random Forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)


In [82]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=2, random_state=0)

In [83]:
y_pred = rf.predict(X_test)
accuracy_score(y_test, y_pred)

0.9585949431528933

In [84]:
from sklearn.metrics import confusion_matrix, classification_report
# confusion_matrix(y_test, y_pred)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.96      1.00      0.98      5649
           1       0.00      0.00      0.00       244

    accuracy                           0.96      5893
   macro avg       0.48      0.50      0.49      5893
weighted avg       0.92      0.96      0.94      5893



/Users/lokki/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lokki/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/lokki/myenv/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [89]:
nodes_df_with_weights_num.to_csv('whole_final_dataset.csv', index=False)